# PyBullet

This is a series of notebooks related to PyBullet, created after following the official guide.

Overview

1. Introduction
    1.1 Hello World
        - Client-server architecture and connection mode
        - Most important functions
        - URDF files
        - GUI control
    1.2 Shapes: Visual & Collision
    1.3 Simulation: `stepSimulation()`, `performCollisionDetection()`
    1.4 Pose Transformations
2. Controlling Robots

## 1. Introduction

Official page: [PyBullet](https://pybullet.org/wordpress/)
Guide: [Quickstart](https://docs.google.com/document/d/10sXEhzFRSnvFcl3XxNGhnD4N2SedqwdAvK3dsihxVUA/)

Features:
- Robotics simulation and machine learning
- Robot models (URDF, etc.) can be loaded
- Forward & inverse kinematics + dynamics available
- Collision detection
- Visualization
- Shared memory, TCP, UDP communication

Installation:
```bash
# conda environment
source/conda activate 3d
# install
pip3 install pybullet --upgrade --user
python3 -m pybullet_envs.examples.enjoy_TF_AntBulletEnv_v0_2017may
python3 -m pybullet_envs.examples.enjoy_TF_HumanoidFlagrunHarderBulletEnv_v1_2017jul
python3 -m pybullet_envs.deep_mimic.testrl --arg_file run_humanoid3d_backflip_args.txt
```

In addition to the python library, I cloned the Bullet repositories, because they contain many examples referenced in the Quickstart guide:

```bash
browser https://github.com/bulletphysics
cd ~/play/git_repositories/foreign
git clone https://github.com/bulletphysics/bullet3.git
git clone https://github.com/bulletphysics/pybullet_robots.git
```

Many examples are available and ready to be executed (open VS Code + select conda:3d + click on play) on:
```bash
~/play/git_repositories/foreign/bullet3/examples/pybullet/examples
```

### 1.1 Hello World

In [1]:
# PyBullet
import pybullet as p
import time
# Sample loader: off-the-shelf samples accessible
import pybullet_data

PyBullet has a Client-Server Architecture: The Physics is the server that serves data requested by a client. Several servers or connection modes are available; the two most common ones are:
- `DIRECT`: only simulation, no graphics nor GUI displayed, but rendered images can be obtained
- `GUI`: simulation + graphics (with GUI)

The graphics run in a separate thread for `GUI` on Windows/Linux, in the same for OSX.

The other connection modes refer to cases where we connect to a server in another process or even on another machine and require additional arguments (such as `hostName` and `port`):
- `SHARED_MEMORY`
- `UDP`
- `TCP_GUI_SERVER`
- `SHARED_MEMORY_SERVER`
- `SHARED_MEMORY_GUI`


In [11]:
# Select client connection mode to physics server: DIRECT, GUI, ...
# On Mac, if GUI selected, the OpenGL visualization is in the same thread...
#physicsClient = p.connect(p.GUI)
physicsClient = p.connect(p.DIRECT)
# Set search path for off-the-shelf samples or set own path with models
p.setAdditionalSearchPath(pybullet_data.getDataPath())
# physicsClientId is optional, for when we connect to several servers
p.setGravity(0,0,-10)
# Loaded from the path added above
# For each object we get an Id handle
# We can add several arguments:
# basePosition, baseOrientation, flags (many options)...
# IMPORTANT parameter: useMaximalCoordinates:
# links are defined as 6DoF rigid bodies
# and constraints defined between them using btMultiBody
# There are also other load interfaces:
# - loadSDF: Simulation Description Format, native Gazebo
# - loadMJCF: MuJoCo, used in OpenAI
planeId = p.loadURDF("plane.urdf")
startPos = [0,0,1]
startOrientation = p.getQuaternionFromEuler([0,0,0])
boxId = p.loadURDF("r2d2.urdf", startPos, startOrientation)
# Set the center of mass frame (loadURDF sets base link frame)
startPosOrn = p.resetBasePositionAndOrientation(boxId, startPos, startOrientation)
for i in range (100): # 10000
    # Forward dynamics simulation:
    # collision detection + constraint solving + integration
    # Default time step is 1/240 seconds
    p.stepSimulation() # optional: physicsClientId
    time.sleep(1./240.)
# Get pose of an object wit its id: Translation + Quaternion
cubePos, cubeOrn = p.getBasePositionAndOrientation(boxId)
print(cubePos,cubeOrn)

(-0.2899307775179597, -0.8812266213889578, 0.9091515263323624) (-0.601940773233314, 0.7518043232060332, -0.07672059866830999, -0.25801456328659933)


In [ ]:
# We can save the state on memory (saveState) and on disk (saveBullet)
# and then restore it with restoreState.
# Anothe roption is saveWorld, but the complete state is not saved
stateId = p.saveState()
p.restoreState(stateId)
p.removeState(stateId)
p.saveBullet("state.bullet")
p.restoreState(fileName="state.bullet")

In [9]:
# Disconnect client from server
# Always run it to clear server & client
p.disconnect()

In [15]:
# Check where the sample models are located
pybullet_data.getDataPath()

'C:\\Users\\msagardia\\AppData\\Roaming\\Python\\Python37\\site-packages\\pybullet_data'

In [16]:
# Object ID
boxId

1

In [17]:
# Physics client
physicsClient

0

In [18]:
# Convert Quaternion -> Rotation matrix (row-major)
p.getMatrixFromQuaternion(cubeOrn)

(0.9999999999999828,
 -1.853128761909076e-07,
 7.994221436957118e-09,
 1.8531287565065205e-07,
 0.9999999999999806,
 6.758070971541131e-08,
 -7.994233960532654e-09,
 -6.758070823397798e-08,
 0.9999999999999977)

In [19]:
# Get information of the physics client
p.getConnectionInfo(physicsClient)

{'isConnected': 0, 'connectionMethod': 0}

#### Connecting several clients

We can start several parallel simulations with `pybullet_utils.bullet_client`.
Examples in:
- [env_bases.py](https://github.com/bulletphysics/bullet3/blob/master/examples/pybullet/gym/pybullet_envs/env_bases.py)
- [multipleScenes.py](
https://github.com/bulletphysics/bullet3/blob/master/examples/pybullet/gym/pybullet_utils/examples/multipleScenes.py)

In [20]:
from pybullet_utils import bullet_client

In [21]:
c = bullet_client.BulletClient(p.GUI)

In [22]:
c.disconnect()

#### URDF files

We define robot systems with them, consisting of **links** and **joints**:
- Each link can be defined as a shape or COLLADA `.dae` mesh file; link pose can be defined too, as well as different properties: visual object, collision object, inertia, etc.
- Each joint can be of a different type: `fixed`, `continuous`, `revolute`, `prismatic`, etc.; each type has its parameters, as well as proerties (e.g., friction, damping).

For concrete examples: [URDF Tutorials](http://wiki.ros.org/urdf/Tutorials)

Simple example: a cylinder and a box, `multipleshapes.urdf`: 

```xml
<?xml version="1.0"?>
<robot name="multipleshapes">
  <link name="base_link">
    <visual>
      <geometry>
        <cylinder length="0.6" radius="0.2"/>
      </geometry>
    </visual>
  </link>

  <link name="right_leg">
    <visual>
      <geometry>
        <box size="0.6 0.1 0.2"/>
      </geometry>
    </visual>
  </link>

  <joint name="base_to_right_leg" type="fixed">
    <parent link="base_link"/>
    <child link="right_leg"/>
  </joint>

</robot>
```

#### GUI

Mouse control:
- Ctrl/Alt + Left Click = Rotate
- Ctrl/Alt + Middel Click = Translate
- Ctrl/Alt + Right Click = Zoom in/out
- Left Click: grab and move an object

Pannels:

### 1.2 Shapes: Visual & Collision

We can create collision and visual shapes with `p.createCollisionShape()` and `p.createVisualShape()`. Then, an object is created with `p.createMultiBody()`. Predefined collision shapes are available:

```bash
p.GEOM_SPHERE
p.GEOM_BOX
p.GEOM_CAPSULE
p.GEOM_CYLINDER
p.GEOM_PLANE
p.GEOM_MESH
p.GEOM_HEIGHTFIELD
```

Shape-specific parameters can be passed to each collision shape: `radius`, `halfExtens`, etc.

We can create an array of collision shapes using `p.createCollisionShapeArray()`.

We can remove an existing collision shape passing its id to `p.removeCollisionShape()`.

The interface `p.createMultiBody()` can be used to create multi-body structures (bodies + constraints/links) or single objects. See more in [createMultiBodyLinks.py](https://github.com/bulletphysics/bullet3/blob/master/examples/pybullet/examples/createMultiBodyLinks.py).

In [28]:
import pybullet as p
import time
import pybullet_data

p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.createCollisionShape(p.GEOM_PLANE)

sphereRadius = 0.05
shift = [0, -0.02, 0]
meshScale = [0.5, 0.5, 0.5]
# Visual shapes are often meshes
# We pass them color an material (.mtl for OBJ) properties
visualShapeId = p.createVisualShape(shapeType=p.GEOM_MESH,
                                    fileName="duck.obj",
                                    rgbaColor=[1, 1, 1, 1],
                                    specularColor=[0.4, .4, 0],
                                    visualFramePosition=shift,
                                    meshScale=meshScale)
#collisionShapeId = p.createCollisionShape(shapeType=p.GEOM_MESH,
#                                          fileName="duck_vhacd.obj",
#                                          collisionFramePosition=shift,
#                                          meshScale=meshScale)
collisionShapeId = p.createCollisionShape(shapeType=p.GEOM_SPHERE,
                                          radius=sphereRadius,
                                          collisionFramePosition=shift)

rangex = 1
rangey = 1
for i in range(rangex):
    for j in range(rangey):
        p.createMultiBody(baseMass=1,
                          baseInertialFramePosition=[0, 0, 0],
                          baseCollisionShapeIndex=collisionShapeId,
                          baseVisualShapeIndex=visualShapeId,
                          basePosition=[((-rangex / 2) + i) * meshScale[0] * 2,
                                        (-rangey / 2 + j) * meshScale[1] * 2, 1],
                          useMaximalCoordinates=True)

p.setGravity(0,0,-10)
startPosOrnSphere = p.resetBasePositionAndOrientation(colSphereId, [0,0,0], [1,0,0,0])
startPosOrnBox = p.resetBasePositionAndOrientation(colSphereId, [0.5,0,0], [1,0,0,0])
for i in range (100): # 10000
    p.stepSimulation()
    time.sleep(1./240.)

In [29]:
# Disconnect client from server
# Always run it to clear server & client
p.disconnect()

#### getMeshData()

There is an experimental function for obtaining mesh information (vertices & co.), but here it seems it's not working?

In [25]:
import pybullet as p
import time
import pybullet_data

p.connect(p.GUI)
boxId = p.loadURDF("cube.urdf", [0,3,2],useMaximalCoordinates = True)
meshData = p.getMeshData(boxId)
print("meshData=",meshData)

meshData= (0, ())


In [26]:
p.disconnect()

### 1.3 Simulation: `stepSimulation()`, `performCollisionDetection()`

Important API functions for the forward dynamics.

`p.stepSimulartion()`: Collision detection + Constraint solving + Motion equation integration.


`p.performCollisionDetection()`: If called instead of `p.stepSimulartion()`, only collisition detection is performed. Then, we can use `p.getContactPoints()`.


`p.setRealTimeSimulation()`: Usually, we need to actively call `p.stepSimulartion()` to make a forward step in the dynamics calculation. If we do `p.setRealTimeSimulation()`, the server itself decides when to make the step using the real-time clock (RTC), and `p.stepSimulartion()` is no longer necessary. That does not work in `p.DIRECT` mode (because the server and the client are in the same thread), but in all others.

### 1.4 Pose Transformations

List of interfaces:

- `p.resetBasePositionAndOrientation()`
- `p.getBasePositionAndOrientation()` 
- `p.getQuaternionFromEuler()` 
- `p.getEulerFromQuaternion()` 
- `p.getMatrixFromQuaternion()` 
- `p.getAxisAngleFromQuaternion()` 
- `p.getDifferenceQuaternion()`
- `p.multiplyTransforms()`
- `p.invertTransform()`


In [57]:
import pybullet as p
import time
import pybullet_data
import math

In [58]:
physicsClient = p.connect(p.DIRECT)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0,0,-10)
planeId = p.loadURDF("plane.urdf")
startPos = [0,0,1]
startOrientation = p.getQuaternionFromEuler([0,0,0])
boxId = p.loadURDF("r2d2.urdf", startPos, startOrientation)
startPosOrn = p.resetBasePositionAndOrientation(boxId, startPos, startOrientation)
for i in range (100): # 10000
    p.stepSimulation()
    time.sleep(1./240.)

In [59]:
# Get Position and Orientation of an Object
# Position = [x,y,z]
# Orientation: by default it is a Quaternion = [x,y,z,w]
# However, we can use other representations for the orientation
# - Euler angles = [yaw, pitch, roll]: getQuaternionFromEuler(), getEulerFromQuaternion()
# - Rotation matrix = [r11, r12, r13, r21, r22, r23, r31, r32, r33]: getMatrixFromQuaternion()
# - Axis & Angle = [ax, ax, az, alpha]: getAxisAngleFromQuaternion
cubePos1, cubeOrn1 = p.getBasePositionAndOrientation(boxId)
print(cubePos1,cubeOrn1)

(0.01743355299305741, -0.026064609592711174, 1.9707092784097282) (0.08780611449535611, 0.03839278492313803, -0.02730737376471416, 0.9950228076085232)


In [62]:
# Reset Position and Orientation of an Object
# Use 
newOrientation = p.getQuaternionFromEuler([0,0,0.5*math.pi])
print(newOrientation)
p.resetBasePositionAndOrientation(boxId, [0,0,0],newOrientation)

(0.0, 0.0, 0.7071067811865476, 0.7071067811865476)


In [63]:
# Check that the pose is now 0
cubePos2, cubeOrn2 = p.getBasePositionAndOrientation(boxId)
print(cubePos2,cubeOrn2)

(0.0, 0.0, 0.0) (0.0, 0.0, 0.7071067811865475, 0.7071067811865476)


In [72]:
# Get different orientation representations
eulerOrn = p.getEulerFromQuaternion(cubeOrn2)
print("Euler = ",eulerOrn)
quatOrn = p.getQuaternionFromEuler(eulerOrn)
print("Quaternion = ",quatOrn)
rotMat = p.getMatrixFromQuaternion(quatOrn)
print("RotMat = ",rotMat)
axisAngle = p.getAxisAngleFromQuaternion(quatOrn)
print("AxisAngle = ",axisAngle)

Euler =  (0.0, -0.0, 1.5707963267948963)
Quaternion =  (0.0, 0.0, 0.7071067811865475, 0.7071067811865477)
RotMat =  (2.220446049250313e-16, -1.0000000000000002, 0.0, 1.0000000000000002, 2.220446049250313e-16, 0.0, 0.0, 0.0, 1.0)
AxisAngle =  ((0.0, 0.0, 0.9999999403953552), 1.5707963705062866)


In [ ]:
# Transformations
# multiplyTransforms: compose transformation poses
# invertTransform: invert transformation poses
# getDifferenceQuaternion: get rotation step from start to end orientation

In [92]:
pos1 = [0,0,0]
rot1 = p.getQuaternionFromEuler([0,0,0.25*math.pi]) # pi/4 = 45
pos2 = [1,1,1]
rot2 = p.getQuaternionFromEuler([0,0,0.5*math.pi]) # pi/2 = 90

In [93]:
rotDiff = p.getDifferenceQuaternion(rot1,rot2)
print(rotDiff)
print(p.getAxisAngleFromQuaternion(rotDiff)[1]*180/math.pi)

(0.0, 0.0, 0.3826833963394165, 0.9238795042037964)
45.00000808242825


In [94]:
# Now we simulate getDifferenceQuaternion by using invertTransform and multiplyTransforms
# NOTE: it seems the translation vector is not correct...?
pos1_inv, rot1_inv = p.invertTransform(pos1,rot1)
print(pos1_inv,rot1_inv)

(-0.0, 0.0, -0.0) (0.0, 0.0, -0.3826834559440613, 0.9238795042037964)


In [97]:
posStep, rotStep = p.multiplyTransforms(pos1_inv, rot1_inv, pos2, rot2)
print(posStep, rotStep)
print(p.getAxisAngleFromQuaternion(rotStep)[1]*180/math.pi)

(1.4142135381698608, -1.1920928955078125e-07, 1.0) (0.0, 0.0, 0.3826833963394165, 0.9238795638084412)
44.99999100695533


In [98]:
p.disconnect()